In [26]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [22]:
DATABASE_URL = "postgresql://localhost/work"
engine = create_engine(DATABASE_URL)
db_cn = psycopg2.connect(DATABASE_URL)

In [33]:
def download_file(url, local_filename):
    full_path = f"data/{local_filename}"
    if os.path.exists(full_path):
        print(f"{full_path} already exists. No need to download.")
        return
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(full_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"{full_path} downloaded")

def load_csv(csv_name, table_name, db_cn):
    if not os.path.exists(csv_name):
        print(f"File does not exist {csv_name}")
        return
    print(f"read csv into dataframe: {csv_name}")
    df = pd.read_csv(csv_name)
    print(f"create table: {table_name}")
    df[0:100].to_sql(table_name, engine, if_exists='replace', index=False, method='multi')
    cursor = db_cn.cursor()
    cursor.execute(f"truncate table {table_name}")
    print(f"load all data into table: {table_name}")
    with open(csv_name, 'r') as f:
        cursor.copy_expert(f'COPY {table_name} FROM stdin WITH CSV HEADER', f)
        db_cn.commit()


In [28]:
# https://data.seattle.gov/Public-Safety/Seattle-Real-Time-Fire-911-Calls/kzjm-xkqj
# https://data.seattle.gov/Public-Safety/SPD-Crime-Data-2008-Present/tazs-3rd5

SEATTLE_API_URI = "https://data.seattle.gov/api/views/"
SEATTLE_API_PARMS = "accessType=DOWNLOAD&bom=true&format=true"
download_file(f"{SEATTLE_API_URI}kzjm-xkqj/rows.csv?{SEATTLE_API_PARMS}", "seattle_fire_data.csv")
download_file(f"{SEATTLE_API_URI}tazs-3rd5/rows.csv?{SEATTLE_API_PARMS}", "seattle_police_data.csv")

data/seattle_fire_data.csv already exists. No need to download.
data/seattle_police_data.csv already exists. No need to download.


In [32]:
load_csv("data/seattle_police_data.csv", "seattle_police_data", psycopg2.connect(DATABASE_URL))